In [1]:
import torch
import argparse
import time
import math
import os, sys
import itertools
import pickle
from tqdm import tqdm
import numpy as np
import os.path as osp

In [2]:
torch.cuda.device_count()

1

In [3]:
def loadpkl(path):
    with open(path, 'rb') as f:
        rst=pickle.load(f)
    return rst

In [5]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

model = AutoModelForSequenceClassification.from_pretrained('cross-encoder/stsb-roberta-base')
tokenizer = AutoTokenizer.from_pretrained('cross-encoder/stsb-roberta-base')

features = tokenizer(['How many people live in Berlin?', 'How many people live in Berlin?'], ['Berlin has a population of 3,520,031 registered inhabitants in an area of 891.82 square kilometers.', 'New York City is famous for the Metropolitan Museum of Art.'],  padding=True, truncation=True, return_tensors="pt")

labels=[1,0]
labels=torch.tensor(labels)

scores = torch.sigmoid(model(**features,labels=labels).logits)
print(scores)

tensor([[0.4870],
        [0.0023]], grad_fn=<SigmoidBackward0>)


In [6]:
model(**features,labels=labels)

SequenceClassifierOutput(loss=tensor(18.9349, grad_fn=<MseLossBackward0>), logits=tensor([[-0.0522],
        [-6.0632]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [8]:
def get_token(query_text,key_text,tokenizer,max_length,query_length):
        # get query and key ids separately
        query=tokenizer(query_text,
                        max_length=max_length,
                        padding=False,
                        truncation=True,
                        )
        # delete last [SEP] and trauncated to query_length
        query_ids=query['input_ids'][:-1][:query_length]
        key=tokenizer(key_text,
                        max_length=max_length,
                        padding=False,
                        truncation=True,
                        )
        key_ids=key['input_ids']
        # replace begining [CLS] with [SEP]
        key_ids[0]=102


        input_ids=query_ids+key_ids
        #trauncated to max_length
        if len(input_ids)>max_length:
            input_ids=input_ids[:max_length]
            input_ids[-1]=102

        attention_mask=[1 for i in range(len(input_ids))]

        token_type_ids=[0 for i in range(len(query_ids)+1)]+[1 for i in range(len(key_ids)-1)]
        if len(token_type_ids)>max_length:
            token_type_ids=token_type_ids[:max_length]

        #padding
        while len(input_ids)<max_length:
            input_ids.append(0)
        while len(attention_mask)<max_length:
            attention_mask.append(0)
        while len(token_type_ids)<max_length:
            token_type_ids.append(0) 
        
        if len(input_ids)>512:
            print(len(input_ids))
            print(len(token_type_ids))
            print(len(attention_mask))
            print('maxlen->',max_length)
        return {'input_ids':input_ids,
                'token_type_ids':token_type_ids,
                'attention_mask':attention_mask,
        }

In [13]:
token1=get_token('How many people live in Berlin [sep]?','Berlin has a population of 3,520,031 registered inhabitants in an area of 891.82 square kilometers.',tokenizer,512,200)

In [14]:
print(token1)

{'input_ids': [0, 6179, 171, 82, 697, 11, 5459, 646, 1090, 642, 742, 116, 102, 26795, 2614, 34, 10, 1956, 9, 155, 6, 31271, 6, 41317, 3382, 24696, 11, 41, 443, 9, 290, 6468, 4, 6551, 3925, 8130, 4, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [24]:
features = tokenizer(['How many people live in Berlin?'], 
                     ['Berlin has a population of 3,520,031 registered inhabitants in an area of 891.82 square kilometers.'],  
                     padding=True, truncation=True,)
print(features)

{'input_ids': [[0, 6179, 171, 82, 697, 11, 646, 1090, 642, 742, 5459, 116, 2, 2, 26795, 2614, 34, 10, 1956, 9, 155, 6, 31271, 6, 41317, 3382, 24696, 11, 41, 443, 9, 290, 6468, 4, 6551, 3925, 8130, 4, 2]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}


In [25]:
sep_token = tokenizer.sep_token

In [26]:
print(sep_token)

</s>
